In [7]:
import pandas as pd
import string
import re
import nltk
import torch
from transformers import BertModel, BertTokenizer
import sklearn
import numpy as np
import tensorflow


c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_all_cleaned = pd.read_csv('data\df_all_cleaned.csv')

In [5]:
df_all_cleaned = df_all_cleaned.drop(['post_feeling', 'post_judging', 'post_sensing'], axis=1)
df_all_cleaned = df_all_cleaned.rename(columns={'post_extrovert': 'post'})
df_all_cleaned

,auhtor_ID,post,extrovert,feeling,judging,sensing
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,1.0,0.0,0.0
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,0.0,0.0,0.0
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,0.0,1.0,0.0
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,1.0,0.0,0.0
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,0.0,1.0,0.0
151,t2_w0842,The technological singularity. And the possibi...,0.0,0.0,1.0,0.0
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,0.0,1.0,0.0
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,0.0,0.0,0.0


In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join the tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20212397\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20212397\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df_all_cleaned['processed_post'] = df_all_cleaned['post'].apply(preprocess_text)
df_all_cleaned

,auhtor_ID,post,extrovert,feeling,judging,sensing,processed_post
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,1.0,0.0,0.0,wear lorna shore shirt alot public lewd long s...
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,0.0,0.0,0.0,id say accurate characterization ni users read...
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,0.0,1.0,0.0,ya know like people home decorations could sav...
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,1.0,0.0,0.0,true tho theyre kinda interesting buuuut issue...
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,0.0,0.0,1.0,yeah thats one things make better objectively ...
...,...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,0.0,1.0,0.0,change profession would inadmissible country p...
151,t2_w0842,The technological singularity. And the possibi...,0.0,0.0,1.0,0.0,technological singularity possibility contribu...
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,0.0,1.0,0.0,dear god man chill im einstein hawking serious...
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,0.0,0.0,0.0,thats fake lib would say human blood water url...


# Classification using SVM

In [12]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [15]:
df_all_cleaned

,auhtor_ID,post,extrovert,feeling,judging,sensing,processed_post
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,1.0,0.0,0.0,wear lorna shore shirt alot public lewd long s...
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,0.0,0.0,0.0,id say accurate characterization ni users read...
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,0.0,1.0,0.0,ya know like people home decorations could sav...
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,1.0,0.0,0.0,true tho theyre kinda interesting buuuut issue...
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,0.0,0.0,1.0,yeah thats one things make better objectively ...
...,...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,0.0,1.0,0.0,change profession would inadmissible country p...
151,t2_w0842,The technological singularity. And the possibi...,0.0,0.0,1.0,0.0,technological singularity possibility contribu...
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,0.0,1.0,0.0,dear god man chill im einstein hawking serious...
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,0.0,0.0,0.0,thats fake lib would say human blood water url...


In [16]:
%%time

df_labels =  pd.DataFrame()
df_labels['extrovert']= df_all_cleaned['extrovert']
df_labels['feeling']= df_all_cleaned['feeling']
df_labels['judging']= df_all_cleaned['judging']
df_labels['sensing']= df_all_cleaned['sensing']

df_labels

CPU times: total: 0 ns
Wall time: 7.98 ms


,extrovert,feeling,judging,sensing
0,1.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
150,0.0,0.0,1.0,0.0
151,0.0,0.0,1.0,0.0
152,0.0,0.0,1.0,0.0
153,1.0,0.0,0.0,0.0


# Splitting dataset into training and test data

In [17]:
# split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(df_all_cleaned['processed_post'], df_labels, test_size=0.2, random_state=42)


In [22]:
# use SVM to classify the data
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# create a pipeline to vectorize the data and then classify it
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SVC(kernel='linear', probability=True))])

# fit the pipeline with the training data
text_clf.fit(X_train, y_train['extrovert'])

# predict the labels on the test data
predictions = text_clf.predict(X_test)

# print the accuracy score
print(accuracy_score(y_test['extrovert'], predictions))

# print the classification report
print(classification_report(y_test['extrovert'], predictions))

# print the confusion matrix
print(confusion_matrix(y_test['extrovert'], predictions))


0.8064516129032258
              precision    recall  f1-score   support

         0.0       0.81      1.00      0.89        25
         1.0       0.00      0.00      0.00         6

    accuracy                           0.81        31
   macro avg       0.40      0.50      0.45        31
weighted avg       0.65      0.81      0.72        31

[[25  0]
 [ 6  0]]


c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

In [23]:
# do the same for the other three labels
text_clf.fit(X_train, y_train['feeling'])
predictions = text_clf.predict(X_test)
print(accuracy_score(y_test['feeling'], predictions))
print(classification_report(y_test['feeling'], predictions))
print(confusion_matrix(y_test['feeling'], predictions))

text_clf.fit(X_train, y_train['judging'])
predictions = text_clf.predict(X_test)
print(accuracy_score(y_test['judging'], predictions))
print(classification_report(y_test['judging'], predictions))
print(confusion_matrix(y_test['judging'], predictions))

text_clf.fit(X_train, y_train['sensing'])
predictions = text_clf.predict(X_test)
print(accuracy_score(y_test['sensing'], predictions))
print(classification_report(y_test['sensing'], predictions))
print(confusion_matrix(y_test['sensing'], predictions))


0.7419354838709677
              precision    recall  f1-score   support

         0.0       0.74      1.00      0.85        23
         1.0       0.00      0.00      0.00         8

    accuracy                           0.74        31
   macro avg       0.37      0.50      0.43        31
weighted avg       0.55      0.74      0.63        31

[[23  0]
 [ 8  0]]


c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

0.4838709677419355
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        16
         1.0       0.48      1.00      0.65        15

    accuracy                           0.48        31
   macro avg       0.24      0.50      0.33        31
weighted avg       0.23      0.48      0.32        31

[[ 0 16]
 [ 0 15]]


c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

0.8709677419354839
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93        27
         1.0       0.00      0.00      0.00         4

    accuracy                           0.87        31
   macro avg       0.44      0.50      0.47        31
weighted avg       0.76      0.87      0.81        31

[[27  0]
 [ 4  0]]


c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

In [24]:
# use grid search to find the best parameters for the SVM classifier
from sklearn.model_selection import GridSearchCV

# create a pipeline to vectorize the data and then classify it
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SVC(kernel='linear', probability=True))])

# set the parameters to be searched
parameters = {'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'tfidf__max_df': [0.5, 0.75, 1.0],
              'tfidf__min_df': [0.0, 0.25, 0.5],
              'clf__C': [0.1, 1.0, 10.0, 100.0]}

# create a grid search object
grid_search = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=5, verbose=1)

# fit the grid search object to the training data
grid_search.fit(X_train, y_train['extrovert'])

# print the best parameters
grid_search.best_params_

# print the best score
grid_search.best_score_

# print the best estimator
grid_search.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits


c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
48 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\20212397\OneDrive - TU Eindhoven\Desktop\Y3\Y3Q2\LangAI\ASS\LanguageAI-Assignment\.venv\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
  

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, min_df=0.25)),
                ('clf', SVC(C=10.0, kernel='linear', probability=True))])

In [25]:
#  use the best estimator to predict the labels on the test data
predictions = grid_search.best_estimator_.predict(X_test)

# print the accuracy score
print(accuracy_score(y_test['extrovert'], predictions))


0.8064516129032258
